In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/fastai/fastai2 
from fastai2.vision.all import *

In [ ]:
# train and test csv
dataset = pd.read_csv("/kaggle/input/traincsv/train.csv")
test = pd.read_csv("/kaggle/input/testcsv/test.csv")
# paths leading to images
train_path = Path("/kaggle/input/train-datas/train/train/")
test_path = Path("/kaggle/input/testdatas/test/test/")

In [ ]:
dataset['filename'] = dataset.apply(lambda x: str(x.category).zfill(2) + '/' + x.filename, axis=1)
dataset

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(dataset, test_size=0.2, random_state = 42, stratify=dataset.category)

In [ ]:
print(train)

In [ ]:
print(val)

In [ ]:
test.head()

In [ ]:
#val.to_csv("meta_labels.csv")

# Pre-processing images

In [ ]:
# Change IMG_SIZE according to your model

IMG_SIZE = 229

item_tfms = [RandomResizedCrop(IMG_SIZE, min_scale=0.75)]
batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats)]
def get_dls_from_df(df):
    df = df.copy()
    options = {
        "item_tfms": item_tfms,
        "batch_tfms": batch_tfms,
        "bs": 32,
    }
    dls = ImageDataLoaders.from_df(df, train_path, shuffle_train=False, **options)
    return dls

In [ ]:
dls = get_dls_from_df(train)
dls.show_batch()

In [ ]:
dls.items

In [ ]:
# Change model whenever you are using different model
learn = cnn_learner(dls, vgg16_bn, metrics=accuracy, progress=True)
learn.fine_tune(4)

In [ ]:
learn.export('/kaggle/working/vgg16.pkl')

In [ ]:
val_dls = dls.test_dl(val)

meta_inputs = learn.get_preds(dl=val_dls, with_decoded=True)

torch.save(meta_inputs[0], "proba_valid_vgg16")

In [ ]:
val_dls.items

In [ ]:
val_dls.show_batch()

In [ ]:
test_images = test.filename.apply(lambda fn: test_path/fn)
test_dl = dls.test_dl(test_images)

In [ ]:
test_dl.show_batch()

In [ ]:
preds = learn.get_preds(dl=test_dl, with_decoded=True)
preds

In [ ]:
torch.save(preds[0], "proba_test_vgg16")

In [ ]:
submission  = test[["filename"]]
submission["category"] = preds[2]

In [ ]:
submission["category"] = submission.category.apply(lambda c: str(c).zfill(2))

In [ ]:
submission.to_csv("vgg16_submission.csv", index=False)